In [14]:
given = """
To get the magic word, interpret each row as a letter using 5–bit encoding.	Of the statements that lie on a main diagonal, less than half are true.	The magic word contains an O.	The magic word contains an H.	None of the corner statements are true.
The sixteen statements on the perimeter of this grid contain exactly as many true statements as the nine statements on the interior of this grid.	Of the statements that are a knight’s move from this one, exactly half are true.	Of the statements that lie on a main diagonal, exactly three are true.	The magic word contains an A.	The magic word contains a T.
More than half the total statements are true.	The magic word contains an L.	Of the statements that are a king’s move from this one, a multiple of three are true.	This column contains exactly as many true statements as the rightmost column.	Of the statements at the midpoint of an edge (including this one), exactly half are true.
Of the statements that are a rook’s move from this one, an even number are true.	The magic word contains a G.	The magic word starts with a vowel.	The magic word contains an S.	Of the statements that are a bishop’s move from this one, none are true.
Exactly two of the corner statements are true.	None of the statements in this column are true.	The magic word has no repeating letters.	The magic word contains a C.	To get the magic word, interpret each column as a letter using 5–bit encoding.
"""

for letter, line in zip('ABCDE', given.strip('\n').split('\n')):
  for i, clue in enumerate(line.split('\t')):
    print('# %s%s: %s' % (letter, i + 1, clue))

# A1: To get the magic word, interpret each row as a letter using 5–bit encoding.
# A2: Of the statements that lie on a main diagonal, less than half are true.
# A3: The magic word contains an O.
# A4: The magic word contains an H.
# A5: None of the corner statements are true.
# B1: The sixteen statements on the perimeter of this grid contain exactly as many true statements as the nine statements on the interior of this grid.
# B2: Of the statements that are a knight’s move from this one, exactly half are true.
# B3: Of the statements that lie on a main diagonal, exactly three are true.
# B4: The magic word contains an A.
# B5: The magic word contains a T.
# C1: More than half the total statements are true.
# C2: The magic word contains an L.
# C3: Of the statements that are a king’s move from this one, a multiple of three are true.
# C4: This column contains exactly as many true statements as the rightmost column.
# C5: Of the statements at the midpoint of an edge (including this one)

In [67]:
import forge
from puzzle.puzzlepedia import puzzlepedia

puzzle = puzzlepedia.parse("""
(row, column) in ({A, B, C, D, E}, {1, 2, 3, 4, 5})
correct in {T, F}

ALL = {
  A1, A2, A3, A4, A5,
  B1, B2, B3, B4, B5,
  C1, C2, C3, C4, C5,
  D1, D2, D3, D4, D5,
  E1, E2, E3, E4, E5,
}
ROWS = [
  [A1, A2, A3, A4, A5],
  [B1, B2, B3, B4, B5],
  [C1, C2, C3, C4, C5],
  [D1, D2, D3, D4, D5],
  [E1, E2, E3, E4, E5],
]
COLUMNS = [
  [A1, B1, C1, D1, E1],
  [A2, B2, C2, D2, E2],
  [A3, B3, C3, D3, E3],
  [A4, B4, C4, D4, E4],
  [A5, B5, C5, D5, E5],
]
N_ALL = len(ALL)
letters = {k: i+1 for i, k in enumerate('abcdefghijklmnopqrstuvwxyz')}
backward = {v: k for k, v in letters.items()}

def n_true(cells):
  return sum([cell.correct == T for cell in cells])

def constrain(cell, stmt):
  return (cell.correct == T) == stmt

def binary(cells):
  return sum([2**i * (cell.correct == T) for i, cell in enumerate(cells)])

# A1: To get the magic word, interpret each row as a letter using 5–bit encoding.
# Assume true?
#A1.correct == T

# A2: Of the statements that lie on a main diagonal, less than half are true.
DIAGONAL = {A1, B2, C3, D4, E5} | {A5, B4, C3, D2, E1}
constrain(A2, n_true(DIAGONAL) <= len(DIAGONAL))

# A3: The magic word contains an O.
# ???

# A4: The magic word contains an H.
# ???

# A5: None of the corner statements are true.
CORNERS = {A1, A5, E1, E5}
constrain(A5, n_true(CORNERS) == 0)

# B1: The sixteen statements on the perimeter of this grid contain exactly as many true statements as the nine statements on the interior of this grid.
TOP = {A1, A2, A3, A4, A5}
LEFT = {A1, B1, C1, D1, E1}
BOTTOM = {E1, E2, E3, E4, E5}
RIGHT = {A5, B5, C5, D5, E5}
PERIMETER = TOP | LEFT | BOTTOM | RIGHT
INTERIOR = ALL - PERIMETER
constrain(B1, n_true(PERIMETER) == n_true(INTERIOR))

# B2: Of the statements that are a knight’s move from this one, exactly half are true.
B2_KNIGHTS = {A4, C4, D3, D1}
constrain(B2, n_true(B2_KNIGHTS) == 2)

# B3: Of the statements that lie on a main diagonal, exactly three are true.
constrain(B3, n_true(DIAGONAL) == 3)

# B4: The magic word contains an A.
# ???

# B5: The magic word contains a T.
# ???

# C1: More than half the total statements are true.
constrain(C1, n_true(ALL) > N_ALL)

# C2: The magic word contains an L.

# C3: Of the statements that are a king’s move from this one, a multiple of three are true.
C3_KING = {B2, B3, B4, C4, D4, D3, D2, C2}
constrain(C3, (n_true(C3_KING) == 3) or (n_true(C3_KING) == 6))

# C4: This column contains exactly as many true statements as the rightmost column.
constrain(C4, n_true(LEFT) == n_true(RIGHT))

# C5: Of the statements at the midpoint of an edge (including this one), exactly half are true.
MIDPOINTS = {A3, C5, E3, C1}

# D1: Of the statements that are a rook’s move from this one, an even number are true.
D1_ROOK = {A1, B1, C1, E1} | {D2, D3, D4, D5}
constrain(D1, n_true(D1_ROOK) & 1 == 0)

# D2: The magic word contains a G.
# D3: The magic word starts with a vowel.
# D4: The magic word contains an S.
# D5: Of the statements that are a bishop’s move from this one, none are true.
D5_BISHOP = {A2, B3, C4, E4}
constrain(D5, n_true(D5_BISHOP) == 0)

# E1: Exactly two of the corner statements are true.
constrain(E1, n_true(CORNERS) == 2)

# E2: None of the statements in this column are true.
E2_COLUMN = {A2, B2, C2, D2, E2}
constrain(E2, n_true(E2_COLUMN) == 0)

# E3: The magic word has no repeating letters.
# E4: The magic word contains a C.
# E5: To get the magic word, interpret each column as a letter using 5–bit encoding.
E5.correct == T

# TODO:
#print(ROWS)
""")

[['A1', 'A2', 'A3', 'A4', 'A5'], ['B1', 'B2', 'B3', 'B4', 'B5'], ['C1', 'C2', 'C3', 'C4', 'C5'], ['D1', 'D2', 'D3', 'D4', 'D5'], ['E1', 'E2', 'E3', 'E4', 'E5']]
[['A1', 'A2', 'A3', 'A4', 'A5'], ['B1', 'B2', 'B3', 'B4', 'B5'], ['C1', 'C2', 'C3', 'C4', 'C5'], ['D1', 'D2', 'D3', 'D4', 'D5'], ['E1', 'E2', 'E3', 'E4', 'E5']]
[['A1', 'A2', 'A3', 'A4', 'A5'], ['B1', 'B2', 'B3', 'B4', 'B5'], ['C1', 'C2', 'C3', 'C4', 'C5'], ['D1', 'D2', 'D3', 'D4', 'D5'], ['E1', 'E2', 'E3', 'E4', 'E5']]
[['A1', 'A2', 'A3', 'A4', 'A5'], ['B1', 'B2', 'B3', 'B4', 'B5'], ['C1', 'C2', 'C3', 'C4', 'C5'], ['D1', 'D2', 'D3', 'D4', 'D5'], ['E1', 'E2', 'E3', 'E4', 'E5']]


In [60]:
letters = {k: i+1 for i, k in enumerate('abcdefghijklmnopqrstuvwxyz')}
backward = {v: k for k, v in letters.items()}

In [61]:
letters, backward

({'a': 1,
  'b': 2,
  'c': 3,
  'd': 4,
  'e': 5,
  'f': 6,
  'g': 7,
  'h': 8,
  'i': 9,
  'j': 10,
  'k': 11,
  'l': 12,
  'm': 13,
  'n': 14,
  'o': 15,
  'p': 16,
  'q': 17,
  'r': 18,
  's': 19,
  't': 20,
  'u': 21,
  'v': 22,
  'w': 23,
  'x': 24,
  'y': 25,
  'z': 26},
 {1: 'a',
  2: 'b',
  3: 'c',
  4: 'd',
  5: 'e',
  6: 'f',
  7: 'g',
  8: 'h',
  9: 'i',
  10: 'j',
  11: 'k',
  12: 'l',
  13: 'm',
  14: 'n',
  15: 'o',
  16: 'p',
  17: 'q',
  18: 'r',
  19: 's',
  20: 't',
  21: 'u',
  22: 'v',
  23: 'w',
  24: 'x',
  25: 'y',
  26: 'z'})